# TOPIC NAME

## Introduction

This guide will show you how to:

* TODO
* TODO

By the end of it, you will TODO!

## Setup

Install dependencies

In [ ]:
pip install neptune-client TODO

## Step 1: TODO

TODO

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
from torch.optim import SGD
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

from ignite.contrib.handlers.neptune_logger import *

LOG_INTERVAL = 10


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)


def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader


train_batch_size = 64
val_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5

train_loader, val_loader = get_data_loaders(train_batch_size, val_batch_size)
model = Net()
device = 'cpu'

if torch.cuda.is_available():
    device = 'cuda'

optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = nn.CrossEntropyLoss()
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

metrics = {
    'accuracy': Accuracy(),
    'loss': Loss(criterion)
}

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
validation_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)


@trainer.on(Events.EPOCH_COMPLETED)
def compute_metrics(engine):
    train_evaluator.run(train_loader)
    validation_evaluator.run(val_loader)


npt_logger = NeptuneLogger(api_token=None,
                           project_name="neptune-ai/pytorch-ignite-integration",
                           name='ignite-mnist-example',
                           params={'train_batch_size': train_batch_size,
                                   'val_batch_size': val_batch_size,
                                   'epochs': epochs,
                                   'lr': lr,
                                   'momentum': momentum})

npt_logger.attach(trainer,
                  log_handler=OutputHandler(tag="training",
                                            output_transform=lambda loss: {'batchloss': loss},
                                            metric_names='all'),
                  event_name=Events.ITERATION_COMPLETED(every=100))

npt_logger.attach(train_evaluator,
                  log_handler=OutputHandler(tag="training",
                                            metric_names=["loss", "accuracy"],
                                            another_engine=trainer),
                  event_name=Events.EPOCH_COMPLETED)

npt_logger.attach(validation_evaluator,
                  log_handler=OutputHandler(tag="validation",
                                            metric_names=["loss", "accuracy"],
                                            another_engine=trainer),
                  event_name=Events.EPOCH_COMPLETED)

npt_logger.attach(trainer,
                  log_handler=OptimizerParamsHandler(optimizer),
                  event_name=Events.ITERATION_COMPLETED(every=100))

npt_logger.attach(trainer,
                  log_handler=WeightsScalarHandler(model),
                  event_name=Events.ITERATION_COMPLETED(every=100))

npt_logger.attach(trainer,
                  log_handler=GradsScalarHandler(model),
                  event_name=Events.ITERATION_COMPLETED(every=100))

# kick everything off
trainer.run(train_loader, max_epochs=epochs)

# log additional information
torch.save(model.state_dict(), 'model.pth')
npt_logger.experiment.log_artifact('model.pth')

npt_logger.close()

## Step 2: Initialize Neptune

Connects your script to Neptune application. 

In [ ]:
import neptune

neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name="shared/onboarding"
)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune project `project_qualified_name`.

---

**Note:** 


Instead of logging data to the public project 'shared/onboarding' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your username to the ``project_qualified_name`` argument of the ``neptune.init()`` method: ``project_qualified_name='YOUR_USERNAME/sandbox``. Keep `/sandbox` at the end, the `sandbox` project that was automatically created for you.

For example:

```python
neptune.init(project_qualified_name='funky_steve/sandbox', 
             api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn',
            )
```

---

## Step 3: Create an experiment

In [ ]:
neptune.create_experiment(name='great-idea')

This opens a new "experiment" namespace in Neptune to which you can log various objects.

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

## Step 4: TODO

TODO

In [ ]:
TODO

---

**Note:**
   
When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running `neptune.stop()`.

If you are running Neptune in regular `.py` scripts it will stop automatically when your code stops running.

---

In [ ]:
neptune.stop()